In [ ]:
# This notebook demonstrates how to calculate the shape score to classify the protein folds ZMPY3D with CuPy.
#
# The original method involved (https://doi.org/10.1371/journal.pcbi.1007970)
#     1) a trimming procedure to preprocess voxels by removing those far from the center and
#     2) a dynamic grid width for individual structures.
#
# These non-essential preprocessing techniques can be improved and are unrelated to this application note's 
# focus on high-performance GPU-based 3D Zernike moments.
#
#
# This notebook primarily consists of the following steps: 
#     1. Install ZMPY3D_CP.
#     2. Define necessary parameters.
#     3. Load precalculated cache.
#     4. Download example PDB data with coordinates.
#     5. Convert coordinate data into a voxel.
#     6. Create a callable function for generating Zernike moments and normalization.
#     7. Obtain the results.
#     8. A command line interface (CLI) example        

In [ ]:
# Install ZMPY3D versions for CuPy.

! pip install ZMPY3D-CP

print(f"It is recommended to restart the Python kernel for the IPython notebook.")

In [ ]:
# Download example data from GitHub using curl
! curl -OJL https://github.com/tawssie/ZMPY3D/raw/main/1WAC_A.txt
! curl -OJL https://github.com/tawssie/ZMPY3D/raw/main/2JL9_A.txt


In [ ]:
import ZMPY3D_CP as z
import cupy as cp
import numpy as np
import pickle

MaxOrder = 20
GridWidth= 1.00
Param=z.get_global_parameter()

# Find the cache_data directory based on the site package location of ZMPY3D.
BinomialCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/BinomialCache.pkl'
with open(BinomialCacheFilePath, 'rb') as file:
    BinomialCachePKL = pickle.load(file)

LogCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/LogG_CLMCache_MaxOrder{:02d}.pkl'.format(MaxOrder)

with open(LogCacheFilePath, 'rb') as file:
    CachePKL = pickle.load(file)

BinomialCache= cp.array(BinomialCachePKL['BinomialCache'],dtype=cp.float64) # 3D matrix

GCache_pqr_linear= cp.array(CachePKL['GCache_pqr_linear'], dtype=cp.int32)
GCache_complex= cp.array(CachePKL['GCache_complex'],dtype=cp.complex128)
GCache_complex_index= cp.array(CachePKL['GCache_complex_index'], dtype=cp.int32)
CLMCache3D= cp.array(CachePKL['CLMCache3D'],dtype=cp.complex128)
CLMCache= cp.array(CachePKL['CLMCache'], dtype=cp.float64)

RotationIndex=CachePKL['RotationIndex']

s_id=cp.array(np.squeeze(RotationIndex['s_id'][0,0])-1, dtype=cp.int32)
n   =cp.array(np.squeeze(RotationIndex['n'] [0,0]), dtype=cp.int32)
l   =cp.array(np.squeeze(RotationIndex['l'] [0,0]), dtype=cp.int32)
m   =cp.array(np.squeeze(RotationIndex['m'] [0,0]), dtype=cp.int32)
mu  =cp.array(np.squeeze(RotationIndex['mu'][0,0]), dtype=cp.int32)
k   =cp.array(np.squeeze(RotationIndex['k'] [0,0]), dtype=cp.int32)
IsNLM_Value=cp.array(np.squeeze(RotationIndex['IsNLM_Value'][0,0])-1, dtype=cp.int32)

print(f"Now using the MaxOrder of {MaxOrder} and the GridWidth of {GridWidth}.")
print(f"Pre-calculated parameters have been loaded successfully.")


In [ ]:
%%time

PDBFileName='./1WAC_A.txt'

# Convert structure data into coordinates
[XYZ,AA_NameList]=z.get_pdb_xyz_ca(PDBFileName)
# Convert coordinates into voxels using precalculated Gaussian densities
ResidueBox=z.get_residue_gaussian_density_cache(Param)
[Voxel3D,Corner]=z.fill_voxel_by_weight_density(XYZ,AA_NameList,Param['residue_weight_map'],GridWidth,ResidueBox[GridWidth])

# Convert the voxel data into a CuPy object
Voxel3D=cp.array(Voxel3D,dtype=cp.float64)

print(f"Converting PDB to 3D voxel grid with NumPy on CPU, then transferring to GPU memory as CuPy objects.")


In [ ]:
%%time

Dimension_BBox_scaled=cp.shape(Voxel3D)
Dimension_BBox_scaled=cp.array(Dimension_BBox_scaled,dtype=cp.int32)

MaxOrder=cp.array(MaxOrder,dtype=cp.int64)

X_sample = cp.arange(Dimension_BBox_scaled[0] + 1, dtype=cp.float64)
Y_sample = cp.arange(Dimension_BBox_scaled[1] + 1, dtype=cp.float64)
Z_sample = cp.arange(Dimension_BBox_scaled[2] + 1, dtype=cp.float64)

# Calculate the volume mass and the center of mass
[VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,X_sample,Y_sample,Z_sample)

# Calculate the weights for sphere sampling
[AverageVoxelDist2Center,MaxVoxelDist2Center]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,cp.array(Param['default_radius_multiplier'], dtype=cp.float64))

# Apply weights to the geometric moments
Sphere_X_sample, Sphere_Y_sample, Sphere_Z_sample=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)

_,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D
                                  ,MaxOrder
                                  ,Sphere_X_sample
                                  ,Sphere_Y_sample
                                  ,Sphere_Z_sample)

# Convert to scaled 3D Zernike moments
ZMoment_scaled,ZMoment_raw=z.calculate_bbox_moment_2_zm(MaxOrder
                                   , GCache_complex
                                   , GCache_pqr_linear
                                   , GCache_complex_index
                                   , CLMCache3D
                                   , SphereBBoxMoment)

ZMList=[]

# Convert the scaled 3D Zernike moments into 3DZD-based descriptors
ZM_3DZD_invariant=z.get_3dzd_121_descriptor(ZMoment_scaled)

ZMList.append(ZM_3DZD_invariant)

# Calculate alternative 3D Zernike moments for specific normalisation orders 2, 3, 4, and 5
MaxTargetOrder2NormRotate=5
for TargetOrder2NormRotate in range(2, MaxTargetOrder2NormRotate+1):
    ABList = z.calculate_ab_rotation(ZMoment_raw.get(), TargetOrder2NormRotate)  
    ABList = cp.array(ABList)
    ZM = z.calculate_zm_by_ab_rotation(ZMoment_raw, BinomialCache, ABList, MaxOrder, CLMCache,s_id,n,l,m,mu,k,IsNLM_Value)
    ZM_mean, _ = z.get_mean_invariant(ZM)
    ZMList.append(ZM_mean)

# MomentInvariant is a vector that describes shape information
MomentInvariant = cp.concatenate([z[~cp.isnan(z)] for z in ZMList])

TotalResidueWeight=z.get_total_residue_weight(AA_NameList,Param['residue_weight_map'])
[Prctile_list,STD_XYZ_dist2center,S,K]=z.get_ca_distance_info(cp.asarray(XYZ))

# GeoDescriptor contains geometric information derived from coordinates
GeoDescriptor = cp.vstack((AverageVoxelDist2Center, TotalResidueWeight, Prctile_list, STD_XYZ_dist2center, S, K))


print(f"Transforming the gridded voxel into 3D Zernike moments with global normalization, 3DZD style, yields 121 descriptors.")
print(f"Additional normalization is also calculated at targets 2, 3, 4, and 5, for Zernike moment descriptor.")
print(f"Geometric information is calculated and collected as a geometric descriptor based on the coordinates.")
print(f"The dimensions of the voxel being used are {Voxel3D.shape}.")
print(f"Time elapsed is as follows:")


In [ ]:
def OneTimeConversion_CP(XYZ,AA_NameList,Voxel3D,MaxOrder):
    Dimension_BBox_scaled=cp.shape(Voxel3D)
    Dimension_BBox_scaled=cp.array(Dimension_BBox_scaled,dtype=cp.int32)
    
    MaxOrder=cp.array(MaxOrder,dtype=cp.int64)
    
    X_sample = cp.arange(Dimension_BBox_scaled[0] + 1, dtype=cp.float64)
    Y_sample = cp.arange(Dimension_BBox_scaled[1] + 1, dtype=cp.float64)
    Z_sample = cp.arange(Dimension_BBox_scaled[2] + 1, dtype=cp.float64)
    
    [VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,X_sample,Y_sample,Z_sample)
    
    [AverageVoxelDist2Center,MaxVoxelDist2Center]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,cp.array(Param['default_radius_multiplier'], dtype=cp.float64))
    
    Sphere_X_sample, Sphere_Y_sample, Sphere_Z_sample=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)
    
    _,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D
                                      ,MaxOrder
                                      ,Sphere_X_sample
                                      ,Sphere_Y_sample
                                      ,Sphere_Z_sample)
    
    ZMoment_scaled,ZMoment_raw=z.calculate_bbox_moment_2_zm(MaxOrder
                                       , GCache_complex
                                       , GCache_pqr_linear
                                       , GCache_complex_index
                                       , CLMCache3D
                                       , SphereBBoxMoment)
    
    ZMList=[]

    ZM_3DZD_invariant=z.get_3dzd_121_descriptor(ZMoment_scaled)
    
    ZMList.append(ZM_3DZD_invariant)
    
    MaxTargetOrder2NormRotate=5
    for TargetOrder2NormRotate in range(2, MaxTargetOrder2NormRotate+1):
        ABList = z.calculate_ab_rotation(ZMoment_raw.get(), TargetOrder2NormRotate)  
        ABList = cp.array(ABList)
        ZM = z.calculate_zm_by_ab_rotation(ZMoment_raw, BinomialCache, ABList, MaxOrder, CLMCache,s_id,n,l,m,mu,k,IsNLM_Value)
        ZM_mean, _ = z.get_mean_invariant(ZM)
        ZMList.append(ZM_mean)

    MomentInvariant = cp.concatenate([z[~cp.isnan(z)] for z in ZMList])
    
    TotalResidueWeight=z.get_total_residue_weight(AA_NameList,Param['residue_weight_map'])
    [Prctile_list,STD_XYZ_dist2center,S,K]=z.get_ca_distance_info(cp.asarray(XYZ))
    
    GeoDescriptor = cp.vstack((AverageVoxelDist2Center, TotalResidueWeight, Prctile_list, STD_XYZ_dist2center, S, K))
    return MomentInvariant, GeoDescriptor


print(f"Merge all steps into a single callable CuPy function, OneTimeConversion_CP.")


In [ ]:
%%time

ResidueBox=z.get_residue_gaussian_density_cache(Param)

# These weights and indexes are predefined in the paper available at https://doi.org/10.1371/journal.pcbi.1007970.
P=z.get_descriptor_property()
ZMIndex  = cp.vstack([P['ZMIndex0'], P['ZMIndex1'], P['ZMIndex2'], P['ZMIndex3'], P['ZMIndex4']])
ZMWeight = cp.vstack([P['ZMWeight0'], P['ZMWeight1'], P['ZMWeight2'], P['ZMWeight3'], P['ZMWeight4']])

# Transforming coordinates into voxels
PDBFileName_A='./1WAC_A.txt'
[XYZ_A,AA_NameList_A]=z.get_pdb_xyz_ca(PDBFileName_A)
[Voxel3D_A,Corner_A]=z.fill_voxel_by_weight_density(XYZ_A,AA_NameList_A,Param['residue_weight_map'],GridWidth,ResidueBox[GridWidth])
Voxel3D_A=cp.array(Voxel3D_A,dtype=cp.float64)

PDBFileName_B='./2JL9_A.txt'
[XYZ_B,AA_NameList_B]=z.get_pdb_xyz_ca(PDBFileName_B)
[Voxel3D_B,Corner_B]=z.fill_voxel_by_weight_density(XYZ_B,AA_NameList_B,Param['residue_weight_map'],GridWidth,ResidueBox[GridWidth])
Voxel3D_B=cp.array(Voxel3D_B,dtype=cp.float64)

# Retrieve descriptors
MomentInvariantRawA, GeoDescriptorA=OneTimeConversion_CP(XYZ_A,AA_NameList_A,Voxel3D_A,20)
MomentInvariantRawB, GeoDescriptorB=OneTimeConversion_CP(XYZ_B,AA_NameList_B,Voxel3D_B,20)


# Computing scores using normalized Zernike moments with specified weights and indices
ZMScore = cp.sum(cp.abs(MomentInvariantRawA[ZMIndex] - MomentInvariantRawB[ZMIndex]) * ZMWeight)

# Computing scores from coordinates using specified weights and indices
GeoScore = cp.sum( cp.asarray(P['GeoWeight']) * (2 * cp.abs(GeoDescriptorA - GeoDescriptorB) / (1 + cp.abs(GeoDescriptorA) + cp.abs(GeoDescriptorB))))


# Rescale scores, where the score is a metric relative to a predefined threshold, not a statistic
GeoScoreScaled = (6.6 - GeoScore) / 6.6 * 100.0
ZMScoreScaled = (9.0 - ZMScore) / 9.0 * 100.0


print(f"Converting two PDB files 1WAC_A.txt and 2JL9_A.txt to gridded voxels, transferring to GPU memory as CuPy objects.")
print(f"Calculate all descriptors for two structures at GridWidth {GridWidth}, deriving the similarity scores.")

print(f'GeoScore {GeoScoreScaled:.2f} TotalZMScore {ZMScoreScaled:.2f}')
print(f"Time elapsed is as follows:")

In [ ]:
# Alternatively, use a system call to compute results via CLI
# ./ZMPY3D_CP_CLI_ShapeScore PDB_A PDB_B GridWidth
! ZMPY3D_CP_CLI_ShapeScore "./1WAC_A.txt" "./2JL9_A.txt" 1.0